# Sberbank Housing - Advanced Preprocessing
## Optimized for Regression

## Imports

In [74]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer, KNNImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
from sklearn.feature_selection import mutual_info_regression
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Load Data & Log-Transform Target

In [19]:
# Load data
data = pd.read_csv(r"data/sberbank_housing.csv", index_col=0, low_memory=False)
data.columns = [c.lower().strip().replace(" ", "_") for c in data.columns]
data = data.drop(columns=["timestamp", "id"])

print(f"Original: {data.shape[0]} samples, {data.shape[1]} features")

# Log-transform target
data['price_doc'] = np.log1p(data['price_doc'])
print("  Target log-transformed")

Original: 27000 samples, 18 features
  Target log-transformed


## Categorical Features Overview

Examine categorical columns to determine encoding strategy based on cardinality.

In [20]:
# Identify categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()

print(f"\nCategorical columns: {len(categorical_cols)}")
print("="*60)

# Initialize lists for cardinality-based categorization
low_card = []
high_card = []
cardinality_threshold = 10

for col in categorical_cols:
    n_unique = data[col].nunique()
    n_missing = data[col].isna().sum()
    missing_pct = (n_missing / len(data)) * 100
    
    # Categorize based on cardinality
    if n_unique < cardinality_threshold:
        low_card.append(col)
        cardinality_label = "LOW"
    else:
        high_card.append(col)
        cardinality_label = "HIGH"
    
    print(f"\n{col}: [{cardinality_label} cardinality]")
    print(f"  Unique values: {n_unique}")
    print(f"  Missing: {n_missing} ({missing_pct:.1f}%)")
    print(f"  Top 5 values:")
    print(data[col].value_counts().head(5).to_string().replace('\n', '\n    '))

print("\nEncoding Strategy:")
print(f"  LOW cardinality (<{cardinality_threshold} unique)  → One-Hot Encoding: {low_card}")
print(f"  HIGH cardinality (≥{cardinality_threshold} unique) → Target/Frequency Encoding: {high_card}")


Categorical columns: 3

product_type: [LOW cardinality]
  Unique values: 2
  Missing: 0 (0.0%)
  Top 5 values:
product_type
    Investment       17542
    OwnerOccupier     9458

ecology: [LOW cardinality]
  Unique values: 5
  Missing: 0 (0.0%)
  Top 5 values:
ecology
    poor            7215
    no data         6570
    good            6548
    excellent       3419
    satisfactory    3248

sub_area: [HIGH cardinality]
  Unique values: 146
  Missing: 0 (0.0%)
  Top 5 values:
sub_area
    Nekrasovka                 1540
    Poselenie Sosenskoe        1522
    Poselenie Vnukovskoe       1048
    Poselenie Moskovskij        807
    Poselenie Voskresenskoe     693

Encoding Strategy:
  LOW cardinality (<10 unique)  → One-Hot Encoding: ['product_type', 'ecology']
  HIGH cardinality (≥10 unique) → Target/Frequency Encoding: ['sub_area']


## Feature Engineering (MI-Guided)

Compute pairwise mutual information (MI) between features, then programmatically create:
- **Ratios & differences** for high MI pairs (>0.5): capture efficiency/gap relationships
- **Interactions** for moderate MI pairs (0.1-0.5): capture non-linear dependencies
- **Polynomials** (degree 2) for top 5 features with highest MI to target

In [21]:
# Calculate Mutual Information matrix for numeric features
numeric_cols = data.select_dtypes(include=[np.number]).columns.drop('price_doc')

# Compute MI for each pair of features
# First, fill NaN values for MI calculation
data_filled = data[numeric_cols].fillna(data[numeric_cols].median())

# Create MI matrix (pairwise MI between features)
mi_matrix = pd.DataFrame(index=numeric_cols, columns=numeric_cols, dtype=float)

for i, col1 in enumerate(numeric_cols):
    for j, col2 in enumerate(numeric_cols):
        if i == j:
            mi_matrix.loc[col1, col2] = 0.0
        elif i < j:
            # Calculate MI between the two features
            mi_val = mutual_info_regression(
                data_filled[[col1]].values,
                data_filled[col2].values,
                random_state=42
            )[0]
            mi_matrix.loc[col1, col2] = mi_val
            mi_matrix.loc[col2, col1] = mi_val
        else:
            continue

# Storage for new features
ratio_features = {}
diff_features = {}
interaction_features = {}

# Analyze MI and create features programmatically
feature_count = {'ratios': 0, 'differences': 0, 'interactions': 0}

for i in range(len(numeric_cols)):
    for j in range(i+1, len(numeric_cols)):
        col1, col2 = numeric_cols[i], numeric_cols[j]
        mi_val = mi_matrix.loc[col1, col2]

        # High MI (> 0.5): Create ratios and differences
        if mi_val > 0.5:
            # Ratio (efficiency/gap measure)
            ratio_name = f'{col1}_div_{col2}'
            ratio_features[ratio_name] = data[col1] / (data[col2] + 1)

            # Difference (gap measure)
            diff_name = f'{col1}_minus_{col2}'
            diff_features[diff_name] = data[col1] - data[col2]

            feature_count['ratios'] += 1
            feature_count['differences'] += 1

        # Moderate MI (0.1 < MI < 0.5): Create interactions
        elif 0.1 < mi_val < 0.5:
            interaction_name = f'{col1}_x_{col2}'
            interaction_features[interaction_name] = data[col1] * data[col2]

            feature_count['interactions'] += 1

# Add all programmatic features
for name, values in ratio_features.items():
    data[name] = values
for name, values in diff_features.items():
    data[name] = values
for name, values in interaction_features.items():
    data[name] = values

print(f"  Created {feature_count['ratios']} ratio features (MI > 0.5)")
print(f"  Created {feature_count['differences']} difference features (MI > 0.5)")
print(f"  Created {feature_count['interactions']} interaction features (0.1 < MI < 0.5)")

  Created 22 ratio features (MI > 0.5)
  Created 22 difference features (MI > 0.5)
  Created 54 interaction features (0.1 < MI < 0.5)


## Polynomial Features

In [22]:
# Select key features for polynomial expansion based on MI with target
numeric_cols_updated = data.select_dtypes(include=[np.number]).columns.drop('price_doc')
data_filled_updated = data[numeric_cols_updated].fillna(data[numeric_cols_updated].median())

# Calculate MI with target for all features
target_mi = mutual_info_regression(
    data_filled_updated.values,
    data['price_doc'].values,
    random_state=42
)

# Create series and sort
mi_series = pd.Series(target_mi, index=numeric_cols_updated).sort_values(ascending=False)

# Top 5 highest MI features get polynomial (degree 2)
top_features = mi_series.head(5).index.tolist()
poly_count = 0

for feat in top_features:
    if feat in data.columns:
        data[f'{feat}_squared'] = data[feat] ** 2
        poly_count += 1

print(f"  Created {poly_count} polynomial features (degree 2 for top MI with target)")

  Created 5 polynomial features (degree 2 for top MI with target)


## SNR Analysis (Signal-to-Noise Ratio)

Calculate SNR = |correlation_with_target| / coefficient_of_variation for each feature. Remove features with SNR < 0.005 (high noise relative to predictive signal).

In [23]:
# Measure Signal-to-Noise Ratio for ALL features (original + engineered)
numeric_cols = data.select_dtypes(include=[np.number]).columns.drop('price_doc')

snr_results = []
for col in numeric_cols:
    valid_mask = data[col].notna()
    if valid_mask.sum() > 100:
        values = data.loc[valid_mask, col]
        target = data.loc[valid_mask, 'price_doc']

        # Calculate correlation as signal
        corr, _ = stats.pearsonr(values, target)
        signal = abs(corr)

        # Standard deviation as noise
        noise = values.std() / values.mean() if values.mean() != 0 else np.inf

        # SNR
        snr = signal / noise if noise > 0 else 0

        snr_results.append({
            'feature': col,
            'signal': signal,
            'noise': noise,
            'snr': snr
        })

snr_df = pd.DataFrame(snr_results).sort_values('snr', ascending=False)

# Remove low SNR features
low_snr = snr_df[snr_df['snr'] < 0.005]['feature'].tolist()
if low_snr:
    data = data.drop(columns=low_snr)
    print(f"  Removed {len(low_snr)} low-SNR features")

  Removed 35 low-SNR features


## VIF Analysis (Multicollinearity Removal)

Identify features with both high correlation (Pearson > 0.95) AND high VIF (>10). Only drop features meeting both criteria to remove redundancy while preserving complementary information.

In [24]:
# Identify highly correlated features (Pearson > 0.95)
numeric_cols = data.select_dtypes(include=[np.number]).columns.drop('price_doc')
corr_matrix = data[numeric_cols].corr().abs()

# Get upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.95
to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.95)]

# Calculate VIF for each feature
X_vif = data[numeric_cols].fillna(data[numeric_cols].median())

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

# Identify features with VIF > 10
high_vif = vif_data[vif_data['VIF'] > 10]['feature'].tolist()

# Only drop features that are in BOTH lists (high correlation AND high VIF)
to_drop_multicollinear = list(set(to_drop_corr).intersection(set(high_vif)))

if to_drop_multicollinear:
    data = data.drop(columns=to_drop_multicollinear)
    print(f"  Removed {len(to_drop_multicollinear)} features (high corr AND high VIF)")
else:
    print(f"  No features with both high correlation AND high VIF")

  Removed 17 features (high corr AND high VIF)


## Train-Test Split

In [25]:
# Split data (do this BEFORE encoding to preserve categorical columns)
X = data.drop('price_doc', axis=1)
y = data['price_doc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save original split with categorical columns for encoding later
X_train_orig = X_train.copy()
X_test_orig = X_test.copy()

## Adaptive Imputation Strategy Selection

Test 4 imputation methods (median, mean, KNN-5, KNN-10) using Ridge regression. Select strategy with lowest test RMSE.

In [26]:
# Test multiple imputation methods and select best
print("Testing imputation strategies...")

numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

imputation_strategies = {
    'median': SimpleImputer(strategy='median'),
    'mean': SimpleImputer(strategy='mean'),
    'knn_5': KNNImputer(n_neighbors=5),
    'knn_10': KNNImputer(n_neighbors=10)
}

imputation_results = []

for strategy_name, imputer in imputation_strategies.items():
    # Create temporary imputed datasets
    X_train_temp = imputer.fit_transform(X_train[numeric_features])
    X_test_temp = imputer.transform(X_test[numeric_features])

    X_train_temp_df = pd.DataFrame(X_train_temp, columns=numeric_features, index=X_train.index)
    X_test_temp_df = pd.DataFrame(X_test_temp, columns=numeric_features, index=X_test.index)

    # Quick model to evaluate
    scaler_temp = RobustScaler()
    X_train_scaled = scaler_temp.fit_transform(X_train_temp_df)
    X_test_scaled = scaler_temp.transform(X_test_temp_df)

    model_temp = Ridge(alpha=1.0, random_state=42)
    model_temp.fit(X_train_scaled, y_train)

    y_pred = model_temp.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    imputation_results.append({
        'strategy': strategy_name,
        'rmse': rmse
    })
    print(f"  {strategy_name:12s}: RMSE = {rmse:.4f}")

# Select best imputation strategy
best_strategy = min(imputation_results, key=lambda x: x['rmse'])
print(f"\n  Selected best: {best_strategy['strategy']} (RMSE: {best_strategy['rmse']:.4f})")

# Apply best imputation
best_imputer = imputation_strategies[best_strategy['strategy']]
X_train_imputed = best_imputer.fit_transform(X_train[numeric_features])
X_test_imputed = best_imputer.transform(X_test[numeric_features])

X_train = pd.DataFrame(X_train_imputed, columns=numeric_features, index=X_train.index)
X_test = pd.DataFrame(X_test_imputed, columns=numeric_features, index=X_test.index)

Testing imputation strategies...
  median      : RMSE = 0.4984
  mean        : RMSE = 0.5030
  knn_5       : RMSE = 0.4855
  knn_10      : RMSE = 0.4854

  Selected best: knn_10 (RMSE: 0.4854)


## Outlier Clipping

In [27]:
# Clip outliers at 1st and 99th percentiles
for col in X_train.columns:
    if X_train[col].dtype in [np.float64, np.int64]:
        lower = X_train[col].quantile(0.01)
        upper = X_train[col].quantile(0.99)
        X_train[col] = X_train[col].clip(lower, upper)
        X_test[col] = X_test[col].clip(lower, upper)

print("  Outliers clipped at 1st and 99th percentiles")

  Outliers clipped at 1st and 99th percentiles


## One-Hot Encoding (Low Cardinality)

In [28]:
# Use the low_card list from categorical overview section
for col in low_card:
    if col in X_train.columns:
        # One-hot encode for train
        train_dummies = pd.get_dummies(X_train[col], prefix=col, drop_first=True)
        X_train = pd.concat([X_train.drop(col, axis=1), train_dummies], axis=1)

        # One-hot encode for test
        test_dummies = pd.get_dummies(X_test[col], prefix=col, drop_first=True)
        X_test = pd.concat([X_test.drop(col, axis=1), test_dummies], axis=1)

        # Align columns
        X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(f"  One-hot encoded {len([c for c in low_card if c in X_train_orig.columns])} low-cardinality features: {[c for c in low_card if c in X_train_orig.columns]}")

  One-hot encoded 2 low-cardinality features: ['product_type', 'ecology']


## Adaptive Encoding Strategy Selection (High Cardinality)

In [29]:
# Test target encoding vs frequency encoding, select best
print("Testing encoding strategies...")

# Use the high_card list from categorical overview section
smoothing = 10

encoding_strategies = {
    'target_encoding': {},
    'frequency_encoding': {}
}

# Test both encoding strategies
for strategy_name in encoding_strategies.keys():
    X_train_temp = X_train.copy()
    X_test_temp = X_test.copy()

    for col in high_card:
        if col in X_train_orig.columns:
            if strategy_name == 'target_encoding':
                # Target encoding with K-Fold
                kf = KFold(n_splits=5, shuffle=True, random_state=42)
                train_encoded = np.zeros(len(X_train_orig))
                global_mean = y_train.mean()

                for train_idx, val_idx in kf.split(X_train_orig):
                    X_fold = X_train_orig.iloc[train_idx]
                    y_fold = y_train.iloc[train_idx]

                    temp_df = pd.DataFrame({col: X_fold[col], 'target': y_fold.values})
                    means = temp_df.groupby(col)['target'].agg(['mean', 'count'])
                    smoothed = (means['mean'] * means['count'] + global_mean * smoothing) / (means['count'] + smoothing)

                    train_encoded[val_idx] = X_train_orig.iloc[val_idx][col].map(smoothed).fillna(global_mean)

                temp_df = pd.DataFrame({col: X_train_orig[col], 'target': y_train.values})
                means = temp_df.groupby(col)['target'].agg(['mean', 'count'])
                smoothed = (means['mean'] * means['count'] + global_mean * smoothing) / (means['count'] + smoothing)
                test_encoded = X_test_orig[col].map(smoothed).fillna(global_mean)

                X_train_temp[f'{col}_enc'] = train_encoded
                X_test_temp[f'{col}_enc'] = test_encoded

            else:  # frequency_encoding
                # Frequency encoding
                freq_map = X_train_orig[col].value_counts(normalize=True).to_dict()
                X_train_temp[f'{col}_enc'] = X_train_orig[col].map(freq_map).fillna(0)
                X_test_temp[f'{col}_enc'] = X_test_orig[col].map(freq_map).fillna(0)

    # Quick model to evaluate
    scaler_temp = RobustScaler()
    X_train_scaled = scaler_temp.fit_transform(X_train_temp)
    X_test_scaled = scaler_temp.transform(X_test_temp)

    model_temp = Ridge(alpha=1.0, random_state=42)
    model_temp.fit(X_train_scaled, y_train)

    y_pred = model_temp.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    encoding_strategies[strategy_name]['rmse'] = rmse
    encoding_strategies[strategy_name]['X_train'] = X_train_temp
    encoding_strategies[strategy_name]['X_test'] = X_test_temp

    print(f"  {strategy_name:20s}: RMSE = {rmse:.4f}")

# Select best encoding strategy
best_encoding = min(encoding_strategies.items(), key=lambda x: x[1]['rmse'])
print(f"\n  Selected best: {best_encoding[0]} (RMSE: {best_encoding[1]['rmse']:.4f})")

# Apply best encoding
X_train = best_encoding[1]['X_train']
X_test = best_encoding[1]['X_test']

encoded_high_card = [c for c in high_card if c in X_train_orig.columns]
print(f"  Applied {best_encoding[0]} for {len(encoded_high_card)} high-cardinality features: {encoded_high_card}")

Testing encoding strategies...
  target_encoding     : RMSE = 0.4669
  frequency_encoding  : RMSE = 0.4766

  Selected best: target_encoding (RMSE: 0.4669)
  Applied target_encoding for 1 high-cardinality features: ['sub_area']


## Outlier Removal via Cook's Distance

Iteratively remove influential outliers using Cook's Distance with a conservative threshold to preserve data while removing extreme leverage points.

In [30]:
from statsmodels.stats.outliers_influence import OLSInfluence
import statsmodels.api as sm

# Conservative Cook's Distance threshold (4/n is standard, we use 4*4/n for conservativeness)
n = len(X_train)
p = X_train.shape[1]
cook_threshold = 4 * (4 / n)  # Very conservative: 16/n instead of 4/n

print(f"Initial training samples: {len(X_train)}")
print(f"Cook's Distance threshold: {cook_threshold:.6f}")

max_iterations = 3  # Limit iterations to prevent over-removal
iteration = 0
outliers_removed_total = 0

while iteration < max_iterations:
    iteration += 1
    
    # Fit OLS model to calculate Cook's Distance
    X_train_with_const = sm.add_constant(X_train)
    model_ols = sm.OLS(y_train, X_train_with_const).fit()
    
    # Calculate Cook's Distance
    influence = OLSInfluence(model_ols)
    cooks_d = influence.cooks_distance[0]
    
    # Identify outliers
    outlier_mask = cooks_d > cook_threshold
    n_outliers = outlier_mask.sum()
    
    if n_outliers == 0:
        print(f"  Iteration {iteration}: No outliers found, stopping")
        break
    
    # Remove outliers from training data
    X_train = X_train[~outlier_mask]
    y_train = y_train[~outlier_mask]
    outliers_removed_total += n_outliers
    
    print(f"  Iteration {iteration}: Removed {n_outliers} outliers (Cook's D > {cook_threshold:.6f})")
    
    # Recalculate n for next iteration
    n = len(X_train)

print(f"\nTotal outliers removed: {outliers_removed_total}")
print(f"Final training samples: {len(X_train)} ({100 * len(X_train) / (len(X_train) + outliers_removed_total):.2f}% retained)")

# Note: X_test remains unchanged (we don't remove test outliers)

Initial training samples: 21600
Cook's Distance threshold: 0.000741
  Iteration 1: Removed 198 outliers (Cook's D > 0.000741)
  Iteration 2: Removed 84 outliers (Cook's D > 0.000741)
  Iteration 3: Removed 42 outliers (Cook's D > 0.000741)

Total outliers removed: 324
Final training samples: 21276 (98.50% retained)


## Robust Scaling

In [31]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_final = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_final = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(f"  Robust scaling applied")

  Robust scaling applied


## Final Model & Evaluation

In [68]:
model = Ridge(alpha=1.0, random_state=42)
model.fit(X_train_final, y_train)

# Predictions
train_pred = model.predict(X_train_final)
test_pred = model.predict(X_test_final)

# Metrics
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
test_r2 = r2_score(y_test, test_pred)

# Convert to RUB
test_pred_rub = np.expm1(test_pred)
y_test_rub = np.expm1(y_test)
test_rmse_rub = np.sqrt(mean_squared_error(y_test_rub, test_pred_rub))

# Cross-validation
cv_scores = cross_val_score(model, X_train_final, y_train, cv=5,
                            scoring='neg_mean_squared_error', n_jobs=-1)
cv_rmse = np.sqrt(-cv_scores).mean()
cv_rmse_std = np.sqrt(-cv_scores).std()

In [72]:
print("\n" + "="*80)
print("RESULTS")
print("="*80)
print(f"Features:       {X_train_final.shape[1]}")
print(f"Samples:        {len(y_train)}")
print(f"\nTest RMSE:      {test_rmse:.4f} (log) | {test_rmse_rub:,.0f} RUB")
print(f"5-Fold CV RMSE MEAN: {cv_rmse:.4f} (log)")
print(f"5-Fold CV RMSE STD: {cv_rmse_std:.4f} (log)")

# Compare with baseline
try:
    with open("baseline_rmse.txt", "r") as f:
        baseline_rmse_rub = float(f.read().strip())
    improvement = ((baseline_rmse_rub - test_rmse_rub) / baseline_rmse_rub) * 100
    print(f"\nBaseline RMSE:  {baseline_rmse_rub:,.0f} RUB")
    print(f"Improvement:    {improvement:+.2f}%")
except:
    pass

print("="*80)


RESULTS
Features:       66
Samples:        21276

Test RMSE:      0.4681 (log) | 3,096,621 RUB
5-Fold CV RMSE MEAN: 0.4473 (log)
5-Fold CV RMSE STD: 0.0084 (log)

Baseline RMSE:  5,626,709 RUB
Improvement:    +44.97%


In [76]:
# import matplotlib.pyplot as plt

# # Calculate residuals
# residuals = y_test - test_pred

# # 1. Check homoscedasticity
# plt.figure(figsize=(12, 5))

# plt.subplot(1, 2, 1)
# plt.scatter(test_pred, residuals, alpha=0.6)
# plt.xlabel("Fitted Values (Predictions)")
# plt.ylabel("Residuals")
# plt.title("Residuals vs Fitted")
# plt.axhline(y=0, color='red', linestyle='--')

# # 2. Check normality of residuals
# plt.subplot(1, 2, 2)
# stats.probplot(residuals, dist="norm", plot=plt)
# plt.title("Q-Q Plot - Residual Normality")

# plt.tight_layout()
# plt.show()

## Results

### TECHNIQUES APPLIED:
1. Programmatic feature engineering (MI-guided):
   - Ratios/differences for MI > 0.5 (efficiency measures)
   - Interactions for 0.1 < MI < 0.5 (non-linear relationships)
   - Polynomial features (degree 2 for top MI with target)

2. SNR analysis (signal-to-noise ratio screening)
3. Multicollinearity removal (Pearson r > 0.95 AND VIF > 10)
4. Adaptive imputation (tested 4 strategies, selected best)
5. Adaptive encoding (target vs frequency, selected best)
6. Outlier clipping (1st and 99th percentiles)
7. Cook's Distance outlier removal (conservative iterative approach, threshold=16/n)
8. Robust scaling (IQR-based)